## Preprocesamiento

In [1]:
import nltk
from nltk.corpus import stopwords
with open('Dataset\stoplist.txt') as file:
    stoplist = [line.lower().strip() for line in file]
stoplist += ['.','¿','?','-','anillo',',',':',';','(',')','«', '»']

def palabras_raiz(texto):
    with open(texto, encoding="utf-8") as file:
        libro = ''
        for line in file:
            libro += line.lower()
    libro_palabras = nltk.word_tokenize(libro)

    libro_palabras_limpias = libro_palabras[:]
    for token in libro_palabras:
        if token in stoplist:
            libro_palabras_limpias.remove(token)

    from nltk.stem.snowball import SnowballStemmer
    stemmer = SnowballStemmer('spanish')
    libro_palabras_raiz = []
    for w in libro_palabras_limpias:
        libro_palabras_raiz.append(stemmer.stem(w))
    return libro_palabras_raiz

libro1_palabras_raiz = palabras_raiz('Dataset\libro1.txt')
libro2_palabras_raiz = palabras_raiz('Dataset\libro2.txt')
libro3_palabras_raiz = palabras_raiz('Dataset\libro3.txt')
libro4_palabras_raiz = palabras_raiz('Dataset\libro4.txt')
libro5_palabras_raiz = palabras_raiz('Dataset\libro5.txt')
libro6_palabras_raiz = palabras_raiz('Dataset\libro6.txt')

from collections import Counter
libro_count_palabras = libro1_palabras_raiz + libro2_palabras_raiz + libro3_palabras_raiz + libro4_palabras_raiz + libro5_palabras_raiz + libro6_palabras_raiz
counter=Counter(libro_count_palabras)
most_common_palabras = sorted([i[0] for i in counter.most_common(500)])

index = {i: [] for i in most_common_palabras} 

for i in index.keys():
    if i in libro1_palabras_raiz:
        index[i].append(1)
    if i in libro2_palabras_raiz:
        index[i].append(2)
    if i in libro3_palabras_raiz:
        index[i].append(3)
    if i in libro4_palabras_raiz:
        index[i].append(4)
    if i in libro5_palabras_raiz:
        index[i].append(5)
    if i in libro6_palabras_raiz:
        index[i].append(6)

with open('invertIndex.txt', 'w') as f:
    for i in index.keys():
        f.write(str(i) + ' : ')
        for j in index[i]:
            f.write(str(j) + ' ')
        f.write('\n')



## InvertIndex

In [2]:
from nltk.stem.snowball import SnowballStemmer

class InvertIndex:
    def __init__(self): 
            self.index = index

    def L(self, key):
        stemmer = SnowballStemmer('spanish')
        key = stemmer.stem(key.lower())
        if key not in self.index:
            return []
        else:
            return self.index[key]
    
    def AND(self, L1, L2):
        return self.intersect(L1, L2)

    def OR(self, L1, L2):
        return self.union(L1, L2)

    def ANDNOT(self, L1, L2):
        return self.difference(L1, L2)
    
    def intersect(self, L1, L2):
        result = []
        p1 = 0
        p2 = 0
        while p1<len(L1) and p2<len(L2):
            if L1[p1] == L2[p2]:
                result.append(L1[p1])
                p1+=1
                p2+=1
            elif L1[p1] < L2[p2]:
                p1+=1
            else:
                p2+=1
        return result

    def union(self, L1, L2):
        result = []
        for i in L1:
            if i not in result:
                result.append(i)
        for i in L2:
            if i not in result:
                result.append(i)
        return result
    
    def difference(self, L1, L2):
        result = []
        for i in L1:
            if i not in result:
                result.append(i)
        for i in L2:
            if i in result:
                result.remove(i)
        return result

retrieve = InvertIndex()
consulta1 = retrieve.OR(retrieve.AND(retrieve.L("Frodo"), retrieve.L("Gandalf")), retrieve.L("Gondor"))
consulta2 = retrieve.ANDNOT(retrieve.OR(retrieve.L("Frodo"), retrieve.L("Gandalf")), retrieve.L("Gondor"))
consulta3 = retrieve.AND(retrieve.OR(retrieve.L("Frodo"), retrieve.L("Gandalf")), retrieve.L("Gondor"))
print("((Frodor AND Gandalf) OR Gondor) aparecen en los libros: ", consulta1)
print("((Frodor OR Gandalf) ANDNOT Gondor) aparecen en los libros: ", consulta2)
print("((Frodor OR Gandalf) AND Gondor) aparecen en los libros: ", consulta3)

((Frodor AND Gandalf) OR Gondor) aparecen en los libros:  [1, 2, 3, 5, 6]
((Frodor OR Gandalf) ANDNOT Gondor) aparecen en los libros:  [1, 4]
((Frodor OR Gandalf) AND Gondor) aparecen en los libros:  [2, 3, 5, 6]
